In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import random
import rf_feature_extraction as rf_feature_extraction
import sys
sys.path.append('../')
from Plotting import plotting_utils as plotting_utils


%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


/usr/local/apps/Anaconda3/2020.11/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
## global variable
datanorm = True
datacols = ['CURRENT_FIX_X', 'CURRENT_FIX_Y', 'CURRENT_FIX_PUPIL', 'CURRENT_FIX_DURATION']

# delta = 10 # size of window based on the middle point
# step =  2*delta+1 #if you want no overlap: 2*delta+1 # size of step in window extraction

datasplit = 'subject'

labelcols = ['subj', 'book',
            'acc_level', 'subj_acc_level', 
            'confidence', 'difficulty', 'familiarity', 'recognition', 
            'interest', 'pressured', 'sleepiness', 'sleephours',
            'sex', 'native']

pred_variable = 'subj_acc_level'

modeltype = 'cnn'

BATCH_SIZE = 100
EPOCHS = 1000

# data path
file_fixdata = '../SB-SAT/fixation/18sat_fixfinal.csv'
file_trialdata = '../SB-SAT/fixation/18sat_trialfinal.csv'
file_labels = '../SB-SAT/fixation/18sat_labels.csv'

In [3]:
## score file load
sc = pd.read_csv(file_labels)

## select label columns
sc = sc[labelcols]
print(sc.loc[sc.subj == 'msd001'])
## cut/replace values
sc['sex'] = sc['sex'].replace(['F', 'M'], [1, 0])
binarycols = ('recognition', 'sex', 'native')
subsetcols = [c for c in labelcols if c not in binarycols]
sc[subsetcols] = sc[subsetcols].replace([0,1,2,3], [0,0,1,1])

     subj       book  acc_level  subj_acc_level  confidence  difficulty  \
0  msd001    dickens          2               3           2           0   
1  msd001    flytrap          3               3           2           1   
2  msd001     genome          3               3           1           0   
3  msd001  northpole          3               3           1           1   

   familiarity  recognition  interest  pressured  sleepiness  sleephours sex  \
0            1            0         2          0           1           2   F   
1            2            0         2          1           1           2   F   
2            1            0         2          1           2           2   F   
3            1            0         1          1           2           2   F   

   native  
0       1  
1       1  
2       1  
3       1  


In [4]:
sc.head()

,subj,book,acc_level,subj_acc_level,confidence,difficulty,familiarity,recognition,interest,pressured,sleepiness,sleephours,sex,native
0,msd001,dickens,1,1,1,0,0,0,1,0,0,1,1,1
1,msd001,flytrap,1,1,1,0,1,0,1,0,0,1,1,1
2,msd001,genome,1,1,0,0,0,0,1,0,1,1,1,1
3,msd001,northpole,1,1,0,0,0,0,0,0,1,1,1,1
4,msd002,dickens,1,0,1,0,0,0,1,0,0,1,0,1


In [5]:
## fixation data load
fd = pd.read_csv(file_fixdata)
fd.head()

,RECORDING_SESSION_LABEL,TRIAL_INDEX,CURRENT_FIX_X,CURRENT_FIX_Y,CURRENT_FIX_PUPIL,CURRENT_FIX_DURATION,CURRENT_FIX_INTEREST_AREA_ID,CURRENT_FIX_INTEREST_AREA_LABEL,CURRENT_FIX_INTEREST_AREA_PIXEL_AREA,CURRENT_FIX_INTEREST_AREA_RUN_ID,...,Trial_Recycled_,total_page,type,book_name,book,page,RT,answer,correct_answer,page_name
0,msd001,1,59.8,125.4,1430.0,22,NaN,NaN,NaN,NaN,...,True,5,reading,dickens,1,1,25094.538413,1,-99,reading-dickens-1
1,msd001,1,348.7,182.0,1375.0,26,24.0,long,3520.0,1.0,...,True,5,reading,dickens,1,1,25094.538413,1,-99,reading-dickens-1
2,msd001,1,630.5,400.3,1365.0,216,72.0,safe,3136.0,1.0,...,True,5,reading,dickens,1,1,25094.538413,1,-99,reading-dickens-1
3,msd001,1,492.0,400.2,1440.0,125,69.0,boundless,7488.0,1.0,...,True,5,reading,dickens,1,1,25094.538413,1,-99,reading-dickens-1
4,msd001,1,526.6,390.5,1265.0,486,70.0,"world,",4992.0,1.0,...,True,5,reading,dickens,1,1,25094.538413,1,-99,reading-dickens-1


In [6]:
fixData = fd.copy(deep=True)

In [7]:
users = fixData.RECORDING_SESSION_LABEL.unique().tolist()
books = fixData.book_name.unique().tolist()
pages = fixData.page.unique().tolist()
data_arr = []#np.empty((0, fixData.shape[1]))
label_arr = np.empty((0, sc.shape[1]))
for user in tqdm(users):
    for book in books:
        tmp_label = sc.loc[sc.subj == user].loc[sc.book == book]
        for page in pages:
            tmp_df = fixData.loc[fixData.RECORDING_SESSION_LABEL == user].loc[fixData.book_name == book].loc[fixData.page == page]
            #XXXXXXX
            tmp_len = len(tmp_df)
            if tmp_len > 0:
                data_arr.append(tmp_df)
                label_arr = np.vstack([label_arr, tmp_label])

In [8]:
data_arr_np = np.array(data_arr)
print(data_arr_np.shape)

(4940,)


<ipython-input-8-df72452750c2>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data_arr_np = np.array(data_arr)


In [9]:
np.unique(np.array(fixData['CURRENT_FIX_INTEREST_AREA_LABEL'],dtype=np.str), return_counts = True)

(array(['(FIN)', '(forms', '(it', ..., '\x93If', '\x93milking',
        '\x93pharming,\x94'], dtype='<U18'),
 array([382, 154,  37, ...,  48, 185, 241]))

In [10]:
np.unique(np.array(fixData['PREVIOUS_SAC_DIRECTION'],dtype=np.str), return_counts = True)

(array(['DOWN', 'LEFT', 'RIGHT', 'UP', 'nan'], dtype='<U5'),
 array([ 14976, 139047, 270115,  17011,  22415]))

In [11]:
data_arr_rf, feature_names_rf = rf_feature_extraction.get_combined_features(data_arr_np)

100%|██████████| 4940/4940 [01:59<00:00, 41.46it/s]


In [12]:
print(data_arr_rf.shape)

(4940, 317)


In [20]:
feature_names_rf

['CURRENT_FIX_X_mean',
 'CURRENT_FIX_X_std',
 'CURRENT_FIX_X_median',
 'CURRENT_FIX_X_skew',
 'CURRENT_FIX_X_kurtosis',
 'CURRENT_FIX_Y_mean',
 'CURRENT_FIX_Y_std',
 'CURRENT_FIX_Y_median',
 'CURRENT_FIX_Y_skew',
 'CURRENT_FIX_Y_kurtosis',
 'CURRENT_FIX_PUPIL_mean',
 'CURRENT_FIX_PUPIL_std',
 'CURRENT_FIX_PUPIL_median',
 'CURRENT_FIX_PUPIL_skew',
 'CURRENT_FIX_PUPIL_kurtosis',
 'CURRENT_FIX_DURATION_mean',
 'CURRENT_FIX_DURATION_std',
 'CURRENT_FIX_DURATION_median',
 'CURRENT_FIX_DURATION_skew',
 'CURRENT_FIX_DURATION_kurtosis',
 'CURRENT_FIX_INTEREST_AREA_ID_mean',
 'CURRENT_FIX_INTEREST_AREA_ID_std',
 'CURRENT_FIX_INTEREST_AREA_ID_median',
 'CURRENT_FIX_INTEREST_AREA_ID_skew',
 'CURRENT_FIX_INTEREST_AREA_ID_kurtosis',
 'CURRENT_FIX_INTEREST_AREA_PIXEL_AREA_mean',
 'CURRENT_FIX_INTEREST_AREA_PIXEL_AREA_std',
 'CURRENT_FIX_INTEREST_AREA_PIXEL_AREA_median',
 'CURRENT_FIX_INTEREST_AREA_PIXEL_AREA_skew',
 'CURRENT_FIX_INTEREST_AREA_PIXEL_AREA_kurtosis',
 'CURRENT_FIX_INTEREST_AREA_RUN_ID_

In [18]:
len(data_arr_np[2])

164

In [19]:
data_arr_np[3]

,RECORDING_SESSION_LABEL,TRIAL_INDEX,CURRENT_FIX_X,CURRENT_FIX_Y,CURRENT_FIX_PUPIL,CURRENT_FIX_DURATION,CURRENT_FIX_INTEREST_AREA_ID,CURRENT_FIX_INTEREST_AREA_LABEL,CURRENT_FIX_INTEREST_AREA_PIXEL_AREA,CURRENT_FIX_INTEREST_AREA_RUN_ID,...,Trial_Recycled_,total_page,type,book_name,book,page,RT,answer,correct_answer,page_name
328,msd001,4,71.8,118.2,1822.0,177,4.0,of,4070.0,1.0,...,True,5,reading,dickens,1,4,19794.729413,1,-99,reading-dickens-4
329,msd001,4,139.9,113.3,1756.0,240,5.0,cigarette,7326.0,1.0,...,True,5,reading,dickens,1,4,19794.729413,1,-99,reading-dickens-4
330,msd001,4,100.8,101.8,1700.0,212,5.0,cigarette,7326.0,1.0,...,True,5,reading,dickens,1,4,19794.729413,1,-99,reading-dickens-4
331,msd001,4,228.5,112.7,1690.0,203,6.0,papers.,6290.0,1.0,...,True,5,reading,dickens,1,4,19794.729413,1,-99,reading-dickens-4
332,msd001,4,232.4,131.8,1749.0,312,6.0,papers.,6290.0,1.0,...,True,5,reading,dickens,1,4,19794.729413,1,-99,reading-dickens-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,msd001,9,519.9,271.3,2107.0,149,NaN,NaN,NaN,NaN,...,False,13,question,dickens,1,4,18566.772413,4,4,question-dickens-4
855,msd001,9,472.7,272.9,2105.0,211,NaN,NaN,NaN,NaN,...,False,13,question,dickens,1,4,18566.772413,4,4,question-dickens-4
856,msd001,9,543.6,264.4,2087.0,346,NaN,NaN,NaN,NaN,...,False,13,question,dickens,1,4,18566.772413,4,4,question-dickens-4
857,msd001,9,466.3,469.2,2075.0,137,NaN,NaN,NaN,NaN,...,False,13,question,dickens,1,4,18566.772413,4,4,question-dickens-4


In [13]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import evaluate_rf as evaluate_rf


folds = 5
rf_accs = []
baseline_accs = []
grid_search_verbosity = 1


aucs = []
fprs = []
tprs = []
best_param_list = []
for f in range(folds):
    # TODO: redo as stratifiedkfold over subjects
    subjkeys = list(set(label_arr[:, 0]))
    # set seeds (maybe define it as utils)
    random.seed(f)
    np.random.seed(f)
    subjkeys = np.random.permutation(subjkeys) #random shuffling
    
    N_totalsub = len(subjkeys)
    N_trainsub = round(0.8*N_totalsub)
    N_validsub = round(0.1*N_totalsub)
    N_testsub = N_totalsub - N_trainsub - N_validsub


    N_train_sub = subjkeys[:N_trainsub]
    N_val_sub = subjkeys[N_trainsub:N_trainsub + N_validsub]
    N_test_sub = subjkeys[N_trainsub + N_validsub: N_trainsub + N_validsub + N_testsub]
    X_train = data_arr_rf[np.isin(label_arr[:, 0], N_train_sub)]
    X_val = data_arr_rf[np.isin(label_arr[:, 0], N_val_sub)]
    X_test = data_arr_rf[np.isin(label_arr[:, 0], N_test_sub)]
    y_train = label_arr[np.isin(label_arr[:, 0], N_train_sub)]
    y_val = label_arr[np.isin(label_arr[:, 0], N_val_sub)]
    y_test = label_arr[np.isin(label_arr[:, 0], N_test_sub)]
        
    # baseline 
    dummy_clf = DummyClassifier(strategy="most_frequent")
    dummy_clf.fit(X_train,np.array(y_train[:, 2], dtype=int))
    
    # rf
    pred_proba, best_parameters, rf = evaluate_rf.evaluate_rf(X_train, X_val, X_test,
                                                          np.array(y_train[:, 2], dtype=int), 
                                                          np.array(y_val[:, 2], dtype=int))
    fpr, tpr, _ = metrics.roc_curve(np.array(y_test[:, 2], dtype=int), pred_proba[:,1], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    #print('auc: ' + str(auc))
    aucs.append(auc)
    fprs.append(fpr)
    tprs.append(tpr)
    
    baseline_accs.append(dummy_clf.score(X_test, np.array(y_test[:, 2], dtype=int)))
    rf_accs.append(rf.score(X_test, np.array(y_test[:, 2], dtype=int)))
    print('baseline: ' + str(baseline_accs[-1]) + ' vs. ' + str(rf_accs[-1]))

baseline: 0.6944444444444444 vs. 0.6944444444444444
baseline: 0.6666666666666666 vs. 0.6666666666666666


KeyboardInterrupt: 

In [ ]:
print(f'baseline: {np.mean(baseline_accs)} +/- {np.std(baseline_accs)}')

In [ ]:
print(f'rf: {np.mean(rf_accs)} +/- {np.std(rf_accs)}')

In [ ]:
plotting_utils.avg_fnr_fpr_curve(fprs, tprs, label = 'RF', plot_random=True,
                    title = None, plot_statistics = False,
                    loc = 'best', plot_legend = True,
                    plot_points = 10000, ncol=1,
                    bbox_to_anchor=None,
                    starting_point = None,
                    fontsize = 14, xscale = None)
plt.title('AUC: ' + str(np.mean(aucs)))
plt.show()

['NOUN', 'NOUN', 'PROPN', 'NOUN', 'NOUN', 'PROPN']

In [394]:
fixation_list = [[100],[],[300,100],[],[200],[300,100],[50]]
fixations_numbers = [[0],[],[1,2],[],[3],[4,5],[6]]
regression_values = [[0],[],[-10,1],[],[1],[3,7],[4]]
word_list = ['ich','Wednesday','Paul','ich','heiße','Paul','nan']
sentence_id_list = [0,0,0,1,1,1,1]
suprisal_list = [0,0,0,1,1,1,1]
pos_tagger = 'spacy'
word_cluster_thresholds = [(1,1),(2,2),(3,3)]

In [395]:
features, feature_names = rf_feature_extraction.get_linguistic_features_for_lists(
                        fixation_list = [[100],[],[300,100],[],[200],[300,100]],
                        fixations_numbers = [[0],[],[1,2],[],[3],[4,5]],
                        regression_values = [[0],[],[-10,1],[],[1],[3,7]],
                        word_list = ['I','am','David','and','I','like'],
                        sentence_id_list = [0,0,0,1,1,1],
                        suprisal_list = [0,0,0,1,1,1,1],
                        pos_tagger = 'spacy',
                        word_cluster_thresholds = [(1,1),(2,2),(3,3)])

In [393]:
features.shape

(49, 6)

In [396]:
features[np.where(np.array(feature_names) == 'dependencies_right')[0],:]

array([[0., 2., 2., 0., 0., 0.]])

In [397]:
features[np.where(np.array(feature_names) == 'dependencies_left')[0],:]

array([[0., 1., 0., 0., 0., 0.]])

In [398]:
len(features)

49

In [399]:
feature_names

['ff',
 'tf',
 'wfc_ff_nomarlized',
 'wfc_ff_normalized',
 'sc_ff_normalized',
 'sc_tf_normalized',
 'ic_ff_normalized',
 'regression',
 'num_regressions',
 'num_progressions',
 'surprisal',
 'word_len',
 'dependencies_right',
 'dependencies_left',
 'is_entity_',
 'is_entity_DATE',
 'is_entity_QUANTITY',
 'is_entity_ORG',
 'is_entity_GPE',
 'is_entity_NORP',
 'is_entity_PERSON',
 'is_entity_TIME',
 'is_entity_CARDINAL',
 'is_entity_ORDINAL',
 'is_entity_NaN',
 'is_pos_PUNCT',
 'is_pos_PROPN',
 'is_pos_NOUN',
 'is_pos_PRON',
 'is_pos_VERB',
 'is_pos_SCONJ',
 'is_pos_NUM',
 'is_pos_DET',
 'is_pos_CCONJ',
 'is_pos_ADP',
 'is_pos_AUX',
 'is_pos_ADV',
 'is_pos_ADJ',
 'is_pos_INTJ',
 'is_pos_X',
 'is_pos_PART',
 'is_content_word_CONTENT',
 'is_content_word_NO_CONTENT',
 'is_content_word_UNKNOWN',
 'is_reduced_pos_ADJ',
 'is_reduced_pos_FUNC',
 'is_reduced_pos_NOUN',
 'is_reduced_pos_UNKNOWN',
 'is_reduced_pos_VERB']

In [366]:
import re
from typing import Any
from typing import List
from typing import Tuple

import pandas as pd
import spacy


def parse_dependency(text) -> Tuple[List[Any], ...]:
    nlp = spacy.load("en_core_web_sm")
    nlp.tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match)
    doc = nlp(text)
    n_rights =  [[] for _ in range(len(text.split()))]
    n_lefts = [[] for _ in range(len(text.split()))]
    rights = [[] for _ in range(len(text.split()))]
    lefts = [[] for _ in range(len(text.split()))]
    deps = [[] for _ in range(len(text.split()))]

    for idx, token in enumerate(doc):
        deps[idx] = token.dep_
        rights[idx] = list(token.rights)
        lefts[idx] = list(token.lefts)
        n_rights[idx] = token.n_rights
        n_lefts[idx] = token.n_lefts
    return deps, n_rights, rights, n_lefts, lefts

In [378]:
word_list = ['I','am','David','and','I','like']
print(parse_dependency(' '.join(word_list)))

(['nsubj', 'ROOT', 'attr', 'cc', 'conj', 'conj'], [0, 2, 2, 0, 0, 0], [[], [David, like], [and, I], [], [], []], [0, 1, 0, 0, 0, 0], [[], [I], [], [], [], []])


In [359]:
np.sum(features[np.where(np.array(feature_names) == 'is_content_word_UNKNOWN')[0],:])

0.0

In [352]:
np.sum(features[np.where(np.array(feature_names) == 'is_entity_DATE')[0],:])

0.0

In [324]:
np.sum(features[np.where(np.array(feature_names) == 'is_entity_')[0],:])

2.0

In [325]:
np.sum(features[np.where(np.array(feature_names) == 'is_entity_ORG')[0],:])

1.0

In [326]:
np.sum(features[np.where(np.array(feature_names) == 'is_entity_NaN')[0],:])

1.0

In [319]:
features.shape

(39, 6)

In [320]:
len(feature_names)

39

In [268]:
feature_names

['ff',
 'tf',
 'wfc_ff_nomarlized',
 'wfc_ff_normalized',
 'sc_ff_normalized',
 'sc_tf_normalized',
 'ic_ff_normalized',
 'regression',
 'num_regressions',
 'num_progressions',
 'surprisal',
 'word_len',
 'is_entity_',
 'is_entity_DATE',
 'is_entity_QUANTITY',
 'is_entity_ORG',
 'is_entity_GPE',
 'is_entity_NORP',
 'is_entity_PERSON',
 'is_entity_TIME',
 'is_entity_CARDINAL',
 'is_entity_ORDINAL',
 'is_entity_NaN',
 'is_pos_PUNCT',
 'is_pos_PROPN',
 'is_pos_NOUN',
 'is_pos_PRON',
 'is_pos_VERB',
 'is_pos_SCONJ',
 'is_pos_NUM',
 'is_pos_DET',
 'is_pos_CCONJ',
 'is_pos_ADP',
 'is_pos_AUX',
 'is_pos_ADV',
 'is_pos_ADJ',
 'is_pos_INTJ',
 'is_pos_X',
 'is_pos_PART']

In [254]:
rf_feature_extraction.get_list_of_pos_tags(word_list)

['NOUN', 'NOUN', 'PROPN', 'NOUN', 'NOUN', 'PROPN', 'NaN']

In [261]:
feature_names

['ff',
 'tf',
 'wfc_ff_nomarlized',
 'wfc_ff_normalized',
 'sc_ff_normalized',
 'sc_tf_normalized',
 'ic_ff_normalized',
 'regression',
 'num_regressions',
 'num_progressions',
 'surprisal']

In [262]:
features

array([[100.        ,   0.        , 300.        ,   0.        ,
        200.        , 300.        ],
       [100.        ,   0.        , 400.        ,   0.        ,
        200.        , 400.        ],
       [  0.5       ,   0.        ,   1.5       ,   0.        ,
          0.8       ,   1.2       ],
       [  0.4       ,   0.        ,   1.6       ,   0.        ,
          0.66666667,   1.33333333],
       [  0.66666667,   0.        ,   1.        ,   0.        ,
          1.33333333,   1.        ],
       [  0.66666667,   0.        ,   1.        ,   0.        ,
          1.33333333,   1.        ],
       [  1.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  -4.5       ,   0.        ,
          1.        ,   5.        ],
       [  0.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   2.        ],
       [  0.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0. 

In [256]:
word_cluster_threshold_ff_fixations

{2: [100]}

In [220]:
feature_matrix

array([[ 1.00000000e+02,  0.00000000e+00,  3.00000000e+02,
         0.00000000e+00,  2.00000000e+02,  3.00000000e+02],
       [ 1.00000000e+02,  0.00000000e+00,  4.00000000e+02,
         0.00000000e+00,  2.00000000e+02,  4.00000000e+02],
       [ 5.00000000e-01,  0.00000000e+00,  1.50000000e+00,
         0.00000000e+00,  8.00000000e-01,  1.20000000e+00],
       [ 4.00000000e-01,  0.00000000e+00,  1.60000000e+00,
         0.00000000e+00,  6.66666667e-01,  1.33333333e+00],
       [ 4.44444444e-01,  0.00000000e+00,  1.33333333e+00,
         0.00000000e+00,  8.88888889e-01,  1.33333333e+00],
       [ 3.63636364e-01,  0.00000000e+00,  1.45454545e+00,
         0.00000000e+00,  7.27272727e-01,  1.45454545e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.50000000e+00,
         0.00000000e+00,  1.00000000e+00,  5.00000000e+00]])

In [190]:
word_cluster_threshold_tf_fixations

{2: [100, 100]}

In [191]:
complete_sentence_tf_fixations

{0: [100, 200, 400], 1: [100, 200, 400]}

In [192]:
complete_sentence_ff_fixations

{0: [100, 200, 300], 1: [100, 200, 300]}

In [173]:
ff
tf
wfc_ff_nomarlized
wfc_tf_normalized
sc_ff_normalized
sc_tf_normalized
ic_ff_normalized
regression

NameError: name 'ff' is not defined

In [63]:
import time
import datetime
import sys
print(time.time())

1642490055.7225032


In [123]:
today = datetime.date.today()
print(today)

2022-01-18


In [127]:
year,month,day = str(today).split('-')
year = int(year)
month = int(month)
day = int(day)

1

In [74]:
today = datetime.datetime.today()
print(today)

2022-01-18 08:26:17.598003


In [75]:
start_time = datetime.datetime(2022,1,18,9)


In [142]:
start_time = datetime.datetime(2022,1,18,9)
end_time = datetime.datetime(2022,1,18,13)
today = datetime.datetime.today()
delta_start = start_time - today
delta_end = end_time - today
if delta_start.seconds < 0 and delta_end.seconds > 0:
    print('run')    
elif delta_start.seconds < 0 and delta_end.seconds < 0:
    sys.exit()
print(delta_start.seconds)
print(delta_end.seconds)


86286
14286


In [144]:
start_time < today and end_time > today

True

In [145]:
start_time < today and  end_time < today

False